In [2]:
using BenchmarkTools, JuMP, Base.Test, ForwardDiff

In [1]:
module acc

using JuMP
using ForwardDiff
using StaticArrays

getconstant(x::AffExpr) = x.constant
getvars(x::AffExpr) = x.vars
getcoeffs(x::AffExpr) = x.coeffs

function accelerate(f::Function, x::AffExpr)
    v = f(x.constant)
    d = ForwardDiff.derivative(f, 0.0)
    AffExpr(copy(x.vars), d * x.coeffs, v)
end

function accelerate(f::Function, xs::Vararg{AffExpr, N}) where N
    v = getconstant.(xs)
    val = f(Tuple(v)...)
    y = zero(SVector{N, Float64})
    d = Tuple(ForwardDiff.gradient(x -> f(Tuple(x)...), y, ForwardDiff.GradientConfig{1}(y)))
    AffExpr(vcat(copy.(getvars.(xs))...), 
        vcat((d .* getcoeffs.(xs))...),
        val)
end


function stable_gradient(f::Function, y::AbstractVector)
    if length(y) < 100
        ForwardDiff.gradient(f, y, ForwardDiff.GradientConfig{1}(y))
    else
        ForwardDiff.gradient(f, y, ForwardDiff.GradientConfig{10}(y))
    end
end

function accelerate_grad(f::Function, v::AbstractVector{AffExpr})
    y = getconstant.(v)
    val = f(y)
    ∇ = stable_gradient(f, y)
    N = sum(x -> length(x.vars), v)
    vars = Vector{Variable}(N)
    coeffs = Vector{Float64}(N)
    offset = 0
    for j in 1:length(v)
        expr = v[j]
        Ni = length(expr.vars)
        for i in 1:Ni
            vars[i + offset] = expr.vars[i]
            coeffs[i + offset] = ∇[j] * expr.coeffs[i]
        end
        offset += Ni
    end
    AffExpr(vars, coeffs, val)
end

accelerate(f::Function, x::AbstractVector{AffExpr}) = accelerate_grad(f, x)

end

acc

In [ ]:
function f(x) 
    y = x - x/2 + 10x - 3x + 10 * x + 1
    for i in 1:100
        y += i * x
    end
    y
end

function g(x)
    y = zero(eltype(x))
    for i in 1:5
        for xx in x
            y += xx
        end
    end
    y
end

h(x, y) = x + y

m = Model()
@variable m q[1:10]
v = [randn(10)' * rand(q, 10) for i in 1:10];
sv = SVector(v...);

setvalue.(q, randn(length(q)))
@test getvalue(f(v[1])) ≈ getvalue(acc.accelerate(f, v[1]))
@test getvalue(g(v)) ≈ getvalue(acc.accelerate(g, v))
# @test getvalue(h(v[1], v[2])) ≈ getvalue(acc.accelerate(h, v[1], v[2]))